# Tradingview web scraper

In [1]:
#import required libraries
import pandas as pd

import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import time
from bs4 import BeautifulSoup
import csv

chromeOptions= Options()
driver = webdriver.Chrome(options=chromeOptions)

current_directory = os.getcwd()
soup = BeautifulSoup(driver.page_source, "html.parser")

#function converting 'hh:mm:ss' time format into seconds
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    total_seconds = h * 3600 + m * 60 + s
    return total_seconds

print(webdriver.__version__)

4.27.1


In [2]:
#if update is needed remove comment below
#pip install --upgrade selenium

In [3]:
#open website and navigate to desired chart
driver.get("https://www.tradingview.com/chart/yTkWcBTE/?symbol=AMEX%3ASPY")

In [4]:
#Importing csv structure to append data
relative_path = 'SPY_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
df = pd.read_csv(csv_file)
time = df['time'].iloc[-1]
df = df.drop(df.index)

#extracting date information, splitting string into individual components
date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
day = date[0]
num = date[1]
month = date[2]
year = date[3][1:]

#main loop for recording data
while 1:
    time_new = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div[2]/span").text
    
    #if new it's a new day reset the date values
    if time_new[:5] == "08:30":
        date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
        day = date[0]
        num = date[1]
        month = date[2]
        year = date[3][1:]
        
    #if time has skipped more than 30 seconds between values, report error and end loop to correct
    if time_to_seconds(time_new) - time_to_seconds(time) > 30:
        print('Error at: ', date, time)
        break
        
    #if value has already been recorded wait until new value is detected
    if time_new == time:
        pass
    
    else:
        time = time_new #sets value as being recorded
        
        #extract text from website and convert to float
        price = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[3]/div[2]/div[1]/div[2]/span").text)
        BBBasis = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[5]/div[2]/div[1]/div[2]/span").text)
        BBUpper = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[5]/div[2]/div[2]/div[2]/span").text)
        BBLower = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[5]/div[2]/div[3]/div[2]/span").text)
        
        #record MFI data and correct syntax if needed
        textarea_element = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[7]/div[2]/div/div[2]/span").text
        if textarea_element == '−0.00': # if syntax is incorrect to be converted to float, correct syntax
            MFI = 0
        else:
            MFI = float(textarea_element)
        
        ATR = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[9]/div[2]/div/div[2]/span").text)
        
        #add placeholder values for TTM and color
        TTM = 0
        color = 0
        
        #record Momentum data
        textarea_element = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[13]/div[2]/div/div[2]/span").text
        if textarea_element == '−0.00':
            Mom = 0
        else:
            Mom = textarea_element
            if Mom[:1] == '−':
                Mom = float('-' + Mom[1:])
            else:
                Mom = float(Mom)
        
        #append new row of data to spreadsheet
        df_new = pd.DataFrame([[year, month, num, day, time, price, BBBasis, BBUpper, BBLower, MFI, ATR, TTM, color, Mom]], columns=df.columns)
        df = pd.concat([df, df_new], ignore_index=False)
        df.to_csv(csv_file, mode='a', header=False, index=False) 
        df = df.drop(df.index) #delete the row of newly added data for data efficiency purposes


KeyboardInterrupt: 

In [5]:
#Importing csv structure to append data
relative_path = 'VOLD_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
df = pd.read_csv(csv_file)
df = df.drop(df.index)

#extracting date information, splitting string into individual components
date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
day = date[0]
num = date[1]
month = date[2]
year = date[3][1:]

#main loop for recording data
while 1: 
    time_new = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div[2]/span").text
    
    #if new it's a new day reset the date values
    if time_new[:2] == "08":
        date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
        day = date[0]
        num = date[1]
        month = date[2]
        year = date[3][1:]
        if date == "Fri 29 Nov '24":
            break
    if time_new == time: # only record new data, idle if data has already been recorded
        pass
    else:
        time = time_new #sets value as being recorded

        #record VOLD data
        VOLD = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[3]/div[2]/div[1]/div[2]/span").text.replace(",", "")
        if VOLD[:1] == '−': #if format is invalid, correct syntax
            VOLD = float('-' + VOLD[1:])
        else:
            VOLD = float(VOLD)

        #append new row of data to spreadsheet
        df_new = pd.DataFrame([[year, month, num, day, time, VOLD]], columns=df.columns)
        df = pd.concat([df, df_new], ignore_index=False)
        df.to_csv(csv_file, mode='a', header=False, index=False)
        df = df.drop(df.index) #delete the row of newly added data for data efficiency purposes
print('Finished.')

ValueError: could not convert string to float: '∅'

In [7]:
#Importing csv structure to append data
relative_path = 'VIX_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
df = pd.read_csv(csv_file)
df = df.drop(df.index)

#extracting date information, splitting string into individual components
date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
day = date[0]
num = date[1]
month = date[2]
year = date[3][1:]

#if time is outside of specified boundaries, no data will be recorded
mode = 0

#main loop for recording data
while 1:
    time_new = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div[2]/span").text
    #if time has entered trading hours, begin recording data for the day
    if time_new[:4] == "08:3": 
        mode = 1
    while mode == 1:
        time_new = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div[2]/span").text
        if time_new[:2] == '15':
            mode = 0
            break
            
        #if new it's a new day reset the date values
        if time_new[:2] == "08":
            date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
            day = date[0]
            num = date[1]
            month = date[2]
            year = date[3][1:]
        if time_new == time: # only record new data, idle if data has already been recorded
            pass
        else:
            time = time_new #sets value as being recorded

            #record VIX data
            VIX = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[3]/div[2]/div[1]/div[2]/span").text)
            VIXBBUpper = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[5]/div[2]/div[2]/div[2]/span").text)
            VIXBBLower = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[5]/div[2]/div[3]/div[2]/span").text)

            #append new row of data to spreadsheet
            df_new = pd.DataFrame([[year, month, num, day, time, VIX, VIXBBUpper, VIXBBLower]], columns=df.columns)
            df = pd.concat([df, df_new], ignore_index=False)
            df.to_csv(csv_file, mode='a', header=False, index=False)
            df = df.drop(df.index)

KeyboardInterrupt: 

In [9]:
#Importing csv structure to append data
relative_path = 'PCC_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
df = pd.read_csv(csv_file)
df = df.drop(df.index)

#extracting date information, splitting string into individual components
date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
day = date[0]
num = date[1]
month = date[2]
year = date[3][1:]

#main loop for recording data
while 1:
    time_new = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div[2]/span").text
    #if new it's a new day reset the date values
    if time_new[:2] == "08":
        date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
        day = date[0]
        num = date[1]
        month = date[2]
        year = date[3][1:]
    if time_new == time: # only record new data, idle if data has already been recorded
        pass
    else:
        time = time_new #sets value as being recorded

        #record data
        PCC = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[3]/div[2]/div[1]/div[2]/span").text

        #append new row of data to spreadsheet
        df_new = pd.DataFrame([[year, month, num, day, time, PCC]], columns=df.columns)
        df = pd.concat([df, df_new], ignore_index=False)
        df.to_csv(csv_file, mode='a', header=False, index=False)
        df = df.drop(df.index)

KeyboardInterrupt: 

In [6]:
#Importing csv structure to append data
relative_path = 'ADD_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
df = pd.read_csv(csv_file)
df = df.drop(df.index)

#extracting date information, splitting string into individual components
date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
day = date[0]
num = date[1]
month = date[2]
year = date[3][1:]

#main loop for recording data
while 1:
    time_new = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div[2]/span").text
    
    #if new it's a new day reset the date values
    if time_new[:2] == "08":
        date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
        day = date[0]
        num = date[1]
        month = date[2]
        year = date[3][1:]
        
    if time_new == time: # only record new data, idle if data has already been recorded
        pass
    else:
        time = time_new #sets value as being recorded

        #record data
        ADD = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[3]/div[2]/div[1]/div[2]/span").text.replace(",", "")
        if ADD[:1] == '−':
            ADD = float('-' + ADD[1:])
        else:
            ADD = float(ADD)

        #append new row of data to spreadsheet
        df_new = pd.DataFrame([[year, month, num, day, time, ADD]], columns=df.columns)
        df = pd.concat([df, df_new], ignore_index=False)
        df.to_csv(csv_file, mode='a', header=False, index=False)
        df = df.drop(df.index)

WebDriverException: Message: target frame detached: received Inspector.detached event
  (Session info: chrome=131.0.6778.265)
Stacktrace:
	GetHandleVerifier [0x00007FF7AE9780D5+2992373]
	(No symbol) [0x00007FF7AE60BFD0]
	(No symbol) [0x00007FF7AE4A590A]
	(No symbol) [0x00007FF7AE4950D1]
	(No symbol) [0x00007FF7AE493EE6]
	(No symbol) [0x00007FF7AE49371D]
	(No symbol) [0x00007FF7AE49363A]
	(No symbol) [0x00007FF7AE491251]
	(No symbol) [0x00007FF7AE491CFF]
	(No symbol) [0x00007FF7AE4A09CE]
	(No symbol) [0x00007FF7AE4B4A9D]
	(No symbol) [0x00007FF7AE4BB26A]
	(No symbol) [0x00007FF7AE492464]
	(No symbol) [0x00007FF7AE4B45EA]
	(No symbol) [0x00007FF7AE53F8CB]
	(No symbol) [0x00007FF7AE51F113]
	(No symbol) [0x00007FF7AE4EA918]
	(No symbol) [0x00007FF7AE4EBA81]
	GetHandleVerifier [0x00007FF7AE9D6A2D+3379789]
	GetHandleVerifier [0x00007FF7AE9EC32D+3468109]
	GetHandleVerifier [0x00007FF7AE9E0043+3418211]
	GetHandleVerifier [0x00007FF7AE76C78B+847787]
	(No symbol) [0x00007FF7AE61757F]
	(No symbol

In [5]:
#Importing csv structure to append data
relative_path = 'TICK_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
df = pd.read_csv(csv_file)
time = df['time'].iloc[-1]
df = df.drop(df.index)

#extracting date information, splitting string into individual components
date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
day = date[0]
num = date[1]
month = date[2]
year = date[3][1:]

#main loop for recording data
while 1:
    time_new = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div[2]/span").text
    
    #if new it's a new day reset the date values
    if time_new[:5] == "08:30":
        date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
        day = date[0]
        num = date[1]
        month = date[2]
        year = date[3][1:]

    if time_to_seconds(time_new) - time_to_seconds(time) > 30:
        print('Error at: ', temp, time_new)
        break
    if time_new == time: # only record new data, idle if data has already been recorded
        pass
    else:
        time = time_new #sets value as being recorded

        #record data
        TICK = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[3]/div[2]/div[1]/div[2]/span").text.replace(",", "")
        if TICK[:1] == '−':
            TICK = float('-' + TICK[1:])
        else:
            TICK = float(TICK)

        TICKBBUpper = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[5]/div[2]/div[2]/div[2]/span").text.replace(",", "")
        if TICKBBUpper[:1] == '−':
            TICKBBUpper = float('-' + TICKBBUpper[1:])
        else:
            TICKBBUpper = float(TICKBBUpper)

        TICKBBLower = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[5]/div[2]/div[3]/div[2]/span").text.replace(",", "")
        if TICKBBLower[:1] == '−':
            TICKBBLower = float('-' + TICKBBLower[1:])
        else:
            TICKBBLower = float(TICKBBLower)

        TICKMA = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[7]/div[2]/div/div[2]/span").text.replace(",", "")
        if TICKMA[:1] == '−':
            TICKMA = float('-' + TICKMA[1:])
        else:
            TICKMA = float(TICKMA)

        #append new row of data to spreadsheet
        df_new = pd.DataFrame([[year, month, num, day, time, TICK, TICKBBUpper, TICKBBLower, TICKMA]], columns=df.columns)
        df = pd.concat([df, df_new], ignore_index=False)
        df.to_csv(csv_file, mode='a', header=False, index=False)
        df = df.drop(df.index)

KeyboardInterrupt: 

In [8]:
#Importing csv structure to append data
relative_path = 'TRIN_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
df = pd.read_csv(csv_file)
df = df.drop(df.index)

#extracting date information, splitting string into individual components
date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
day = date[0]
num = date[1]
month = date[2]
year = date[3][1:]

#main loop for recording data
while 1:
    time_new = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div[2]/span").text
    
    #if new it's a new day reset the date values
    if time_new[:2] == "08":
        date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
        day = date[0]
        num = date[1]
        month = date[2]
        year = date[3][1:]
    if time_new == time: # only record new data, idle if data has already been recorded
        pass
    else:
        time = time_new #sets value as being recorded

        #record data
        TRIN = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[3]/div[2]/div[1]/div[2]/span").text

        #append new row of data to spreadsheet
        df_new = pd.DataFrame([[year, month, num, day, time, TRIN]], columns=df.columns)
        df = pd.concat([df, df_new], ignore_index=False)
        df.to_csv(csv_file, mode='a', header=False, index=False)
        df = df.drop(df.index)

KeyboardInterrupt: 

In [11]:
#Importing csv structure to append data
relative_path = 'SPY5_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
df = pd.read_csv(csv_file)
df = df.drop(df.index)

#extracting date information, splitting string into individual components
date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
day = date[0]
num = date[1]
month = date[2]
year = date[3][1:]

#main loop for recording data
while 1:
    time_new = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div[2]/span").text
    
    #if new it's a new day reset the date values
    if time_new == "08:30":
        date = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div[2]/span").text.split()
        day = date[0]
        num = date[1]
        month = date[2]
        year = date[3][1:]
        a = 0
    if time_new == time: # only record new data, idle if data has already been recorded
        pass
    else:
        time = time_new #sets value as being recorded

        price = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[3]/div[2]/div[1]/div[2]/span").text)
        BBBasis = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[5]/div[2]/div[1]/div[2]/span").text)
        BBUpper = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[5]/div[2]/div[2]/div[2]/span").text)
        BBLower = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[5]/div[2]/div[3]/div[2]/span").text)

        MFI = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[7]/div[2]/div/div[2]/span")
        if MFI.text == '−0.00':
            MFI = 0
        else:
            MFI = float(MFI.text)

        ATR = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[9]/div[2]/div/div[2]/span").text)

        TTM = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[11]/div[2]/div[1]/div[2]/span").text
        if TTM != '−0.00':
            if TTM[:1] == '−':
                TTM = float('-' + TTM[1:])
            else:
                TTM = float(TTM)
        else:
            TTM = 0
            
        if driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[11]/div[2]/div[2]/div[2]/span").value_of_css_property('color') == 'rgba(255, 0, 0, 1)':
            color = 1
        else:
            color = 0
        
        Mom = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[13]/div[2]/div/div[2]/span").text
        if Mom != '−0.00':
            if Mom[:1] == '−':
                Mom = float('-' + Mom[1:])
            else:
                Mom = float(Mom)
        else:
            Mom = 0
                
        WaveASlow = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[17]/div[2]/div[1]/div[2]/span").text
        if WaveASlow != '−0.00':
            if WaveASlow[:1] == '−':
                WaveASlow = float('-' + WaveASlow[1:])
            else:
                WaveASlow = float(WaveASlow)
        else:
            WaveASlow = 0

        WaveAFast = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[17]/div[2]/div[2]/div[2]/span").text
        if WaveAFast != '−0.00':
            if WaveAFast[:1] == '−':
                WaveAFast = float('-' + WaveAFast[1:])
            else:
                WaveAFast = float(WaveAFast)
        else:
            WaveAFast = 0
            
        WaveBSlow = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[19]/div[2]/div[1]/div[2]/span").text
        if WaveBSlow != '−0.00':
            if WaveBSlow[:1] == '−':
                WaveBSlow = float('-' + WaveBSlow[1:])
            else:
                WaveBSlow = float(WaveBSlow)
        else:
            WaveBSlow = 0

        WaveBFast = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[19]/div[2]/div[2]/div[2]/span").text
        if WaveBFast == '−0.00':
            if WaveBFast[:1] == '−':
                WaveBFast = float('-' + WaveBFast[1:])
            else:
                WaveBFast = float(WaveBFast)
        else:
            WaveBFast = 0

        WaveCMCAD = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[21]/div[2]/div[1]/div[2]/span").text
        if WaveCMCAD != '−0.00':
            if WaveCMCAD[:1] == '−':
                WaveCMCAD = float('-' + WaveCMCAD[1:])
            else:
                WaveCMCAD = float(WaveCMCAD)
        else:
            WaveCMCAD = 0
            
        WaveCFast = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[21]/div[2]/div[2]/div[2]/span").text
        if WaveCFast == '−0.00':
            if WaveCFast[:1] == '−':
                WaveCFast = float('-' + WaveCFast[1:])
            else:
                WaveCFast = float(WaveCFast)
        else:
            WaveCFast = 0
        
        df_new = pd.DataFrame([[year, month, num, day, time, price, BBBasis, BBUpper, BBLower, MFI, ATR, TTM, color, Mom, WaveASlow, WaveAFast, WaveBSlow, WaveBFast, WaveCMCAD, WaveCFast]], columns=df.columns)
        df = pd.concat([df, df_new], ignore_index=False)
        df.to_csv(csv_file, mode='a', header=False, index=False)
        df = df.drop(df.index)
        
    if time_new == '14:50' and a == 0:
        textarea_element = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[3]/div[2]/div[1]/div[2]/span")
        price_new = float(textarea_element.text)
        if price_new != price:
            a = 1
            price = price_new
            time = '14:55'
            BBBasis = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[5]/div[2]/div[1]/div[2]/span").text)
            BBUpper = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[5]/div[2]/div[2]/div[2]/span").text)
            BBLower = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[5]/div[2]/div[3]/div[2]/span").text)

            MFI = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[7]/div[2]/div/div[2]/span")
            if MFI.text == '−0.00':
                MFI = 0
            else:
                MFI = float(MFI.text)

            ATR = float(driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[9]/div[2]/div/div[2]/span").text)

            TTM = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[11]/div[2]/div[1]/div[2]/span").text
            if TTM != '−0.00':
                if TTM[:1] == '−':
                    TTM = float('-' + TTM[1:])
                else:
                    TTM = float(TTM)
            else:
                TTM = 0
            
            if driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[11]/div[2]/div[2]/div[2]/span").value_of_css_property('color') == 'rgba(255, 0, 0, 1)':
                color = 1
            else:
                color = 0
        
            Mom = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[13]/div[2]/div/div[2]/span").text
            if Mom != '−0.00':
                if Mom[:1] == '−':
                    Mom = float('-' + Mom[1:])
                else:
                    Mom = float(Mom)
            else:
                Mom = 0
                
            WaveASlow = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[17]/div[2]/div[1]/div[2]/span").text
            if WaveASlow != '−0.00':
                if WaveASlow[:1] == '−':
                    WaveASlow = float('-' + WaveASlow[1:])
                else:
                    WaveASlow = float(WaveASlow)
            else:
                WaveASlow = 0

            WaveAFast = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[17]/div[2]/div[2]/div[2]/span").text
            if WaveAFast != '−0.00':
                if WaveAFast[:1] == '−':
                    WaveAFast = float('-' + WaveAFast[1:])
                else:
                    WaveAFast = float(WaveAFast)
            else:
                WaveAFast = 0
            
            WaveBSlow = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[19]/div[2]/div[1]/div[2]/span").text
            if WaveBSlow != '−0.00':
                if WaveBSlow[:1] == '−':
                    WaveBSlow = float('-' + WaveBSlow[1:])
                else:
                    WaveBSlow = float(WaveBSlow)
            else:
                WaveBSlow = 0

            WaveBFast = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[19]/div[2]/div[2]/div[2]/span").text
            if WaveBFast == '−0.00':
                if WaveBFast[:1] == '−':
                    WaveBFast = float('-' + WaveBFast[1:])
                else:
                    WaveBFast = float(WaveBFast)
            else:
                WaveBFast = 0

            WaveCMCAD = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[21]/div[2]/div[1]/div[2]/span").text
            if WaveCMCAD != '−0.00':
                if WaveCMCAD[:1] == '−':
                    WaveCMCAD = float('-' + WaveCMCAD[1:])
                else:
                    WaveCMCAD = float(WaveCMCAD)
            else:
                WaveCMCAD = 0
                
            WaveCFast = driver.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[1]/div[3]/div/div[2]/div/div[2]/div/div[21]/div[2]/div[2]/div[2]/span").text
            if WaveCFast == '−0.00':
                if WaveCFast[:1] == '−':
                    WaveCFast = float('-' + WaveCFast[1:])
                else:
                    WaveCFast = float(WaveCFast)
            else:
                WaveCFast = 0
        
            df_new = pd.DataFrame([[year, month, num, day, time, price, BBBasis, BBUpper, BBLower, MFI, ATR, TTM, color, Mom, WaveASlow, WaveAFast, WaveBSlow, WaveBFast, WaveCMCAD, WaveCFast]], columns=df.columns)
            df = pd.concat([df, df_new], ignore_index=False)
            df.to_csv(csv_file, mode='a', header=False, index=False)
            df = df.drop(df.index)
            time = '14:50'

KeyboardInterrupt: 